![alternative text](../../data/colab_chatgpt.png)
![alternative text](../../data/colab2_chatgpt.png)
![alternative text](../../data/colab3_chatgpt.png)


![alternative text](../../data/cont_chatgpt.png)
![alternative text](../../data/cont2_chatgpt.png)
![alternative text](../../data/cont3_chatgpt.png)

![alternative text](../../data/whybrid_chatgpt.png)
![alternative text](../../data/shybrid_chatgpt.png)



In [143]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split


# Content Based Filtering 

In [144]:
# https://www.kaggle.com/datasets/shubhammehta21/movie-lens-small-latest-dataset
# Load MovieLens dataset
movies  = pd.read_csv('../../data/movies.csv')
ratings = pd.read_csv('../../data/ratings.csv')


In [145]:
def cosine_similarity(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    cosine_sim = np.dot(vectors, vectors.T) / (norms * norms.T)
    return cosine_sim

In [146]:
# Merge movies and ratings data
movie_ratings = pd.merge(ratings, movies, on='movieId')
movie_ratings

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


In [147]:
# Pivot the data to create a user-item rating matrix and change no rating to zero
user_movie_ratings = movie_ratings.pivot_table(index='userId', columns='title', values='rating').fillna(0)
user_movie_ratings

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.5,3.5,0.0,0.0,0.0


In [148]:
# Convert user-movie ratings to a numpy array
ratings_matrix = user_movie_ratings.values

# Calculate the cosine similarity between movies
cosine_sim = cosine_similarity(ratings_matrix.T)

# Convert cosine similarity matrix to a DataFrame
cosine_sim_df = pd.DataFrame(cosine_sim, index=user_movie_ratings.columns, columns=user_movie_ratings.columns)
cosine_sim_df


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.141653,0.000000,...,0.000000,0.342055,0.543305,0.707107,0.0,0.000000,0.139431,0.327327,0.000000,0.0
'Hellboy': The Seeds of Creation (2004),0.000000,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Round Midnight (1986),0.000000,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Salem's Lot (2004),0.000000,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Til There Was You (1997),0.000000,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.211467,0.216295,0.097935,0.132489,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.192259,0.000000,0.170341,0.0
xXx (2002),0.139431,0.000000,0.000000,0.000000,0.000000,0.0,0.089634,0.000000,0.276512,0.019862,...,0.069716,0.305535,0.173151,0.246482,0.0,0.192259,1.000000,0.270034,0.100396,0.0
xXx: State of the Union (2005),0.327327,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.156764,0.000000,...,0.000000,0.382543,0.177838,0.231455,0.0,0.000000,0.270034,1.000000,0.000000,0.0


In [149]:
# Recommend movies for a user based on content
def content_based_recommendations_1(movie_title, num_recommendations=5):
    items  = cosine_sim_df[movie_title].sort_values(ascending=False)
    idx    = items.index[1:num_recommendations+1]
    values = items[1:num_recommendations+1]
    recommended_movies = idx.tolist()
    return [(recommended_movie,value) for recommended_movie,value in zip(recommended_movies,values)]

In [150]:
# Say for this user watched "Toy Story (1995)" and other similar users who watched this also watched the recommended movies
recommended_movies = content_based_recommendations_1("Toy Story (1995)")
print("Recommended movies similar to Toy Story:")
for movie_title in recommended_movies:
    print(movie_title)


Recommended movies similar to Toy Story:
('Toy Story 2 (1999)', 0.5726012603197154)
('Jurassic Park (1993)', 0.5656368040861566)
('Independence Day (a.k.a. ID4) (1996)', 0.5642616935276659)
('Star Wars: Episode IV - A New Hope (1977)', 0.5573881705799366)
('Forrest Gump (1994)', 0.5470959079401742)


In [151]:
# Extract genres as features for content-based filtering
vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
genre_matrix = vectorizer.fit_transform(movies['genres']).toarray()

# Calculate cosine similarity between movies based on genres
cosine_sim_genre = cosine_similarity(genre_matrix)
cosine_sim_genre_df = pd.DataFrame(cosine_sim_genre, index=movies['title'], columns=movies['title'])
cosine_sim_genre_df

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),1.000000,0.774597,0.316228,0.258199,0.447214,0.000000,0.316228,0.632456,0.000000,0.258199,...,0.447214,0.316228,0.316228,0.447214,0.0,0.670820,0.774597,0.00000,0.316228,0.447214
Jumanji (1995),0.774597,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.816497,0.000000,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.0,0.288675,0.333333,0.00000,0.000000,0.000000
Grumpier Old Men (1995),0.316228,0.000000,1.000000,0.816497,0.707107,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.353553,0.000000,0.500000,0.000000,0.0,0.353553,0.408248,0.00000,0.000000,0.707107
Waiting to Exhale (1995),0.258199,0.000000,0.816497,1.000000,0.577350,0.000000,0.816497,0.000000,0.000000,0.000000,...,0.288675,0.408248,0.816497,0.000000,0.0,0.288675,0.333333,0.57735,0.000000,0.577350
Father of the Bride Part II (1995),0.447214,0.000000,0.707107,0.577350,1.000000,0.000000,0.707107,0.000000,0.000000,0.000000,...,0.500000,0.000000,0.707107,0.000000,0.0,0.500000,0.577350,0.00000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Black Butler: Book of the Atlantic (2017),0.670820,0.288675,0.353553,0.288675,0.500000,0.288675,0.353553,0.000000,0.500000,0.288675,...,0.750000,0.353553,0.353553,0.500000,0.0,1.000000,0.866025,0.00000,0.707107,0.500000
No Game No Life: Zero (2017),0.774597,0.333333,0.408248,0.333333,0.577350,0.000000,0.408248,0.000000,0.000000,0.000000,...,0.577350,0.408248,0.408248,0.577350,0.0,0.866025,1.000000,0.00000,0.408248,0.577350
Flint (2017),0.000000,0.000000,0.000000,0.577350,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.707107,0.707107,0.000000,0.0,0.000000,0.000000,1.00000,0.000000,0.000000


In [152]:
# Recommend movies for a user based on content
def content_based_recommendations_2(movie_title, num_recommendations=5):
    items  = cosine_sim_genre_df[movie_title].sort_values(ascending=False)
    idx    = items.index[1:num_recommendations+1]
    values = items[1:num_recommendations+1]
    recommended_movies = idx.tolist()
    return [(recommended_movie,value) for recommended_movie,value in zip(recommended_movies,values)]

In [153]:
recommended_movies = content_based_recommendations_2("Toy Story (1995)")
recommended_movies

[('Adventures of Rocky and Bullwinkle, The (2000)', 0.9999999999999998),
 ("Emperor's New Groove, The (2000)", 0.9999999999999998),
 ('Monsters, Inc. (2001)', 0.9999999999999998),
 ('Tale of Despereaux, The (2008)', 0.9999999999999998),
 ('Wild, The (2006)', 0.9999999999999998)]

# Colaborative Filtering 

In [237]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error


In [238]:
# Convert user-item matrix to a PyTorch tensor
user_movie_tensor = torch.tensor(user_movie_ratings.values, dtype=torch.float32)

# Split data into train and test sets
train_tensor, test_tensor = train_test_split(user_movie_tensor, test_size=0.2, random_state=42)

# Define a collaborative filtering model
class CollaborativeFiltering(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super(CollaborativeFiltering, self).__init__()
        self.user_embed = nn.Embedding(num_users, embedding_dim)
        self.item_embed = nn.Embedding(num_items, embedding_dim)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_bias = nn.Embedding(num_items, 1)
    
    def forward(self, user_ids, item_ids):
        user_embeds = self.user_embed(user_ids)
        item_embeds = self.item_embed(item_ids)
        user_bias = self.user_bias(user_ids)
        item_bias = self.item_bias(item_ids)
        dot_product = (user_embeds * item_embeds).sum(dim=1)
        return dot_product + user_bias.squeeze() + item_bias.squeeze()

In [239]:
# Instantiate the collaborative filtering model
num_users, num_items = user_movie_tensor.shape
embedding_dim = 20
collab_model = CollaborativeFiltering(num_users, num_items, embedding_dim)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(collab_model.parameters(), lr=0.001)

# Create DataLoader for training
train_user_ids, train_item_ids = torch.where(train_tensor > 0)
train_dataset = TensorDataset(train_user_ids, train_item_ids, train_tensor[train_user_ids, train_item_ids])
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [240]:
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(collab_model.parameters(), lr=0.001)
num_epochs = 50
for epoch in range(num_epochs):
    for batch_user_ids, batch_item_ids, batch_ratings in train_loader:
        optimizer.zero_grad()
        batch_predictions = collab_model(batch_user_ids, batch_item_ids)
        loss = criterion(batch_predictions, batch_ratings)
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [5/50], Loss: 16.9516
Epoch [10/50], Loss: 5.7540
Epoch [15/50], Loss: 2.5043
Epoch [20/50], Loss: 0.7921
Epoch [25/50], Loss: 1.1496
Epoch [30/50], Loss: 0.4655
Epoch [35/50], Loss: 0.3587
Epoch [40/50], Loss: 0.4067
Epoch [45/50], Loss: 0.3530
Epoch [50/50], Loss: 0.3298


In [241]:
# Evaluate the model on the test set
test_user_ids, test_item_ids = torch.where(test_tensor > 0)
test_predictions = collab_model(test_user_ids, test_item_ids)
test_rmse = np.sqrt(mean_squared_error(test_predictions.detach().numpy(), test_tensor[test_user_ids, test_item_ids]))
test_rmse

2.5374262

# Hybrid 

In [284]:
# Hybrid recommendation model
class HybridRecommendation(nn.Module):
    def __init__(self, collaborative_model, content_similarity):
        super(HybridRecommendation, self).__init__()
        self.collaborative_model = collaborative_model
        self.content_similarity = content_similarity
    
    def forward(self, user_ids, item_ids):
        collab_scores = self.collaborative_model(user_ids, item_ids)
        content_scores = torch.tensor(self.content_similarity[item_ids, :].max(axis=1), dtype=torch.float32)
        hybrid_scores = collab_scores + content_scores
        return hybrid_scores


In [289]:
# Instantiate the collaborative filtering model
embedding_dim = 20
collab_model = CollaborativeFiltering(num_users, num_items, embedding_dim)

# Convert cosine similarity matrix to PyTorch tensor
content_similarity_tensor = torch.tensor(cosine_sim_genre, dtype=torch.float32)

# Instantiate the hybrid recommendation model
hybrid_model = HybridRecommendation(collab_model, content_similarity_tensor)


num_epochs = 20
for epoch in range(num_epochs):
    optimizer.zero_grad()
    user_ids, item_ids = torch.where(train_tensor > 0)
    predictions = hybrid_model(user_ids, item_ids)
    loss = criterion(predictions, train_tensor[user_ids, item_ids])
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


ValueError: only one element tensors can be converted to Python scalars

In [246]:
# Evaluate the model on the test set
test_user_ids, test_item_ids = torch.where(test_tensor > 0)
test_predictions = hybrid_model(test_user_ids, test_item_ids)
test_rmse = np.sqrt(mean_squared_error(test_predictions.detach().numpy(), test_tensor[test_user_ids, test_item_ids]))
print(f"Test RMSE: {test_rmse:.4f}")

Test RMSE: 2.4691


/var/folders/yr/_5d8vnxd6z58c56nmqh8nykw0000gn/T/ipykernel_54709/2811556258.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  content_scores = torch.tensor(self.content_similarity[item_ids, :].max(axis=1).values, dtype=torch.float32)
